In [2]:
import glob
import os
import re
import datetime
import numpy as np
#import xarray as xr
#from netCDF4 import Dataset
import sys

In [23]:
import yaml
with open("config_write_BCs.yml", "r") as f:
    config = yaml.safe_load(f)

In [24]:
start_time_of_interest = np.datetime64(datetime.datetime.strptime(config["startdate"], "%Y%m%d"))
end_time_of_interest = np.datetime64(datetime.datetime.strptime(config["enddate"], "%Y%m%d")) + np.timedelta64(20, 'D')

In [25]:
start_time_of_interest

numpy.datetime64('2018-12-31T00:00:00.000000')

In [26]:
end_time_of_interest

numpy.datetime64('2019-01-22T00:00:00.000000')

In [27]:
def get_TROPOMI_times(filename):
    
    """
    Function that parses the TROPOMI filenames to get the start and end times.
    Example input (str): S5P_RPRO_L2__CH4____20220725T152751_20220725T170921_24775_03_020400_20230201T100624.nc
    Example output (tuple): (np.datetime64('2022-07-25T15:27:51'), np.datetime64('2022-07-25T17:09:21'))
    """

    file_times = re.search(r'(\d{8}T\d{6})_(\d{8}T\d{6})', filename)
    assert file_times is not None, "check TROPOMI filename - wasn't able to find start and end times in the filename"
    start_TROPOMI_time = np.datetime64(datetime.datetime.strptime(file_times.group(1), "%Y%m%dT%H%M%S"))
    end_TROPOMI_time = np.datetime64(datetime.datetime.strptime(file_times.group(2), "%Y%m%dT%H%M%S"))
    
    return start_TROPOMI_time, end_TROPOMI_time

In [28]:
TROPOMI_files = sorted([file for file in glob.glob(os.path.join(config["tropomi_cache"], "*.nc"))
                            if (start_time_of_interest <= get_TROPOMI_times(file)[0] <= end_time_of_interest)
                            and (start_time_of_interest <= get_TROPOMI_times(file)[1] <= end_time_of_interest)])

In [29]:
len(TROPOMI_files)

311

In [31]:
assert len(
    [
        file for file in TROPOMI_files
        if (
            start_time_of_interest <= get_TROPOMI_times(file)[1] <= end_time_of_interest - np.timedelta64(20, 'D')
        )
    ]
) > 20, "There doesn't seem to be many files past the last day you want BCs. This is a problem because of the +/- 15 day temporal smoothing."

In [32]:
start_time_of_interest <= get_TROPOMI_times(TROPOMI_files[-1])[1]

True

In [33]:
get_TROPOMI_times(TROPOMI_files[-1])[1]

numpy.datetime64('2019-01-21T22:21:46.000000')

In [34]:
end_time_of_interest - np.timedelta64(20, 'D')

numpy.datetime64('2019-01-02T00:00:00.000000')

In [35]:
[
        file for file in TROPOMI_files
        if (
            start_time_of_interest <= get_TROPOMI_times(file)[1] <= end_time_of_interest - np.timedelta64(20, 'D')
        )
    ]

['/n/holylfs05/LABS/jacob_lab/imi/ch4/blended/S5P_BLND_L2__CH4____20181231T001500_20181231T015630_06294_03_020400_20230614T040525.nc',
 '/n/holylfs05/LABS/jacob_lab/imi/ch4/blended/S5P_BLND_L2__CH4____20181231T015630_20181231T033759_06295_03_020400_20230614T040429.nc',
 '/n/holylfs05/LABS/jacob_lab/imi/ch4/blended/S5P_BLND_L2__CH4____20181231T033759_20181231T051929_06296_03_020400_20230614T040516.nc',
 '/n/holylfs05/LABS/jacob_lab/imi/ch4/blended/S5P_BLND_L2__CH4____20181231T051929_20181231T070059_06297_03_020400_20230614T040443.nc',
 '/n/holylfs05/LABS/jacob_lab/imi/ch4/blended/S5P_BLND_L2__CH4____20181231T070059_20181231T084229_06298_03_020400_20230614T040655.nc',
 '/n/holylfs05/LABS/jacob_lab/imi/ch4/blended/S5P_BLND_L2__CH4____20181231T084229_20181231T102359_06299_03_020400_20230614T040449.nc',
 '/n/holylfs05/LABS/jacob_lab/imi/ch4/blended/S5P_BLND_L2__CH4____20181231T102359_20181231T120529_06300_03_020400_20230614T040659.nc',
 '/n/holylfs05/LABS/jacob_lab/imi/ch4/blended/S5P_BLND_